In [1]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib

In [2]:
plt.rcParams['text.usetex'] = True
plt.rcParams['pgf.rcfonts'] = False
plt.rcParams['pgf.texsystem'] = 'pdflatex'
plt.rcParams['pgf.preamble'] = '\n'.join([
    r'\usepackage[T1, T2A]{fontenc}',
    r'\usepackage[utf8]{inputenc}',
    r'\usepackage[english, russian]{babel}'
    ])
plt.rc('font', family='serif')
plt.switch_backend('pgf')

In [3]:
my_colors = ['green', 'red', 'gray', 'blue', 'purple', 'orange']
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=my_colors)

In [4]:
plt.rcParams['font.size']=14

In [5]:
def stationary_distr(Q):
  evals, evecs = np.linalg.eig(Q.T)
  evec1 = evecs[:,np.isclose(evals, 1)]

#Since np.isclose will return an array, we've indexed with an array
#so we still have our 2nd axis.  Get rid of it, since it's only size 1.
  evec1 = evec1[:,0]

  stationary = evec1 / evec1.sum()

#eigs finds complex eigenvalues and eigenvectors, so you'll want the real part.
  stationary = stationary.real
  return stationary

In [6]:
def init_X(pi):
  U = np.random.uniform(0, 1, 1)
  F = [np.sum(pi[0:i]) for i in range(1, pi.size + 1)]
  for i in range(0, len(F)):
    if F[i] > U:
      return int(i)
  return int(len(F) - 1)

In [7]:
def scale(row):
  s = np.sum(row)
  return row/s

In [8]:
def model(dimensions, markov_chain, discretization):
  dim_X, dim_Y = dimensions
  Lambda, pi, f, g = markov_chain
  H, h = discretization

  # state
  X_grid_length = int(T/H)
  X_grid = np.arange(0, X_grid_length)
  X = np.empty(X_grid_length, dtype = np.int8)
  X[0] = init_X(pi)
  for i in tqdm(X_grid[1:]):
    P = np.eye(dim_X) + H * Lambda
    P_i = P[X[i - 1]]
    X[i] = init_X(scale(P_i))

  # observations
  Y_grid_length = int(T/h)
  Y_grid = np.arange(0, Y_grid_length)
  Y = np.empty((Y_grid_length, dim_Y))
  for i in tqdm(Y_grid):
    W = np.random.multivariate_normal(np.zeros(dim_Y), h * g[X[i * int(h/H)]])  #
    Y[i] = f[X[i * int(h/H)]] * h + W
  
  # filtration
  est_grid_length = Y_grid_length
  estimation = np.empty((est_grid_length, dim_X), dtype = np.dtype('f8'))
  est_grid = np.arange(0, est_grid_length)
  
  def N(Y, F, G):
    return multivariate_normal.pdf(Y, F, G)

  # initial state
  estimation_0 = np.array([1/dim_X] * (dim_X - 1))
  estimation_0 = np.append(estimation_0, 1 - np.sum(estimation_0))

  estimation[0] = estimation_0
  for i in tqdm(est_grid[1:]):
    forecast = (np.eye(dim_X) + h * Lambda.T) @ estimation[i - 1]
    k_t = np.diag([N(Y[i], H * f[j], H * g[j]) for j in range(dim_X)])
    ones = np.ones((1, dim_X))
    estimation[i] = 1 / (ones @ k_t @ forecast) * k_t @ forecast
  
  return X, Y, estimation, X_grid, Y_grid, est_grid

In [9]:
def model_X(dim_X, markov_chain, H, T):
  Lambda, pi, f, g = markov_chain

  # state
  X_grid_length = int(T/H)
  X_grid = np.arange(0, X_grid_length)
  X = np.empty(X_grid_length, dtype = np.int8)
  X[0] = init_X(pi)
  for i in tqdm(X_grid[1:]):
    P = np.eye(dim_X) + H * Lambda
    P_i = P[X[i - 1]]
    X[i] = init_X(scale(P_i))
  
  return X, X_grid

In [10]:
def model_Y(X, dim_Y, discretization, T):
  H, h = discretization

  # observations
  Y_grid_length = int(T/h)
  Y_grid = np.arange(0, Y_grid_length)
  Y = np.empty((Y_grid_length, dim_Y))
  for i in tqdm(Y_grid):
    W = np.random.multivariate_normal(np.zeros(dim_Y), h * g[X[i * int(h/H)]])  #
    Y[i] = f[X[i * int(h/H)]] * h + W  

  return Y, Y_grid  

In [11]:
def filtering(Y, dimensions, markov_chain, h, T):
  # filtration
  dim_X, dim_Y = dimensions
  Lambda, pi, f, g = markov_chain

  est_grid_length = int(T/H)
  estimation = np.empty((est_grid_length, dim_X), dtype = np.dtype('f8'))
  est_grid = np.arange(0, est_grid_length)
  
  N = multivariate_normal.pdf


  estimation[0] = pi#estimation_0
  for i in tqdm(est_grid[1:]):
    forecast = (np.eye(dim_X) + H * Lambda.T) @ estimation[i - 1]
    estimation[i] = forecast
    if i % (int(h/H)) == 0:
        k_t = np.diag([N(Y[i], H * f[j], H * g[j]) for j in range(dim_X)])
        ones = np.ones((1, dim_X))
        estimation[i] = 1 / (ones @ k_t @ forecast) * k_t @ forecast
  return estimation, est_grid

In [12]:
def to_sums(Y, Y_grid):
  Y_grid_length = len(Y_grid)
  sum_Y = np.empty((Y_grid_length, dim_Y))
  sum_Y[0][0] = 1
  for i in tqdm(Y_grid[1:]):
    sum_Y[i] = sum_Y[i - 1] + Y[i - 1]
  return sum_Y

In [13]:
H = 10 ** (-7) # X time
h = 10 ** (-4) # Y time
T = 1 # in minutes
seed = 100
if seed != 0:
    np.random.seed(seed)

dim_X = 4
dim_Y = 1

Lambda = np.array([[-12.5, 12.5, 0, 0],
                    [0, -1000, 1000, 0],
                    [0, 0, -250, 250]
                   ,[40, 0, 10, -50]])
pi = stationary_distr(np.eye(dim_X) - H * Lambda)
f = np.array([[0.07], 
              [0.03], 
              [0.02],
              [0.025]])
g = np.array([np.diag([0.1]), 
              np.diag([0.5]), 
              np.diag([0.6]),
              np.diag([0.3])])


In [14]:
with open('XandY.npy', 'rb') as file:
    npzfile = np.load(file)
    X = npzfile['X'].copy()
    X_grid = npzfile['X_grid'].copy()
    Y = npzfile['Y'].copy()
    Y_grid = npzfile['Y_grid'].copy()
    Y_sum = npzfile['Y_sum'].copy()

In [15]:
pi

array([0.72072072, 0.00900901, 0.04504505, 0.22522523])

In [16]:
markov_chain = (Lambda, pi, f, g)
discretization = (H, h)
dimensions = (dim_X, dim_Y)

#X, X_grid = model_X(dim_X, markov_chain, H, T)
#Y, Y_grid = model_Y(X, dim_Y, discretization, T)
X_pred, est_grid = filtering(Y, dimensions, markov_chain, h, T)
#Y_sum = to_sums(Y, Y_grid)

100%|████████████████████████████████████████████████████████████████████| 9999999/9999999 [01:04<00:00, 154326.57it/s]


In [17]:
argmax = np.argmax(X_pred, axis=1)

In [18]:
np.sum(argmax == X) * H

0.7754521

In [19]:
Y=0

In [20]:
"""plt.figure(figsize=(12, 7), dpi=240)
#plt.title("Результат фильтрации по дискретизованным наблюдениям")
plt.plot(np.arange(0,1,H), X+1, label='Действительное значение', zorder=2)
plt.plot(np.arange(0,1,H), argmax+1, label='Оценка', zorder=1)
plt.legend(loc='upper right')
plt.ylabel('Состояние МСП')
plt.xlabel('Время (год)')
plt.yticks([1, 2, 3, 4])
plt.savefig("filtration_discrete.pgf", bbox_inches='tight')
plt.show()"""

'plt.figure(figsize=(12, 7), dpi=240)\n#plt.title("Результат фильтрации по дискретизованным наблюдениям")\nplt.plot(np.arange(0,1,H), X+1, label=\'Действительное значение\', zorder=2)\nplt.plot(np.arange(0,1,H), argmax+1, label=\'Оценка\', zorder=1)\nplt.legend(loc=\'upper right\')\nplt.ylabel(\'Состояние МСП\')\nplt.xlabel(\'Время (год)\')\nplt.yticks([1, 2, 3, 4])\nplt.savefig("filtration_discrete.pgf", bbox_inches=\'tight\')\nplt.show()'

In [21]:
h = H

In [22]:
"""xstart=0.15
xend=0.2

coor_xstart = int(xstart / h)
coor_xend = int(xend/h)

plt.figure(figsize=(15, 10), dpi=240)
#plt.title("Результат фильтрации по дискретизованным наблюдениям")
plt.plot(np.arange(0,1,h)[coor_xstart:coor_xend], (X+1)[coor_xstart:coor_xend], label='Действительное значение', zorder=2)
plt.plot(np.arange(0,1,h)[coor_xstart:coor_xend], (argmax+1)[coor_xstart:coor_xend], label='Оценка', zorder=1)
plt.legend(loc='upper right')
plt.ylabel('Состояние МСП')
plt.xlabel('Время (год)')
plt.yticks([1, 2, 3, 4])
plt.savefig("filtration_zoom_discrete.pgf", bbox_inches='tight')
#plt.show()"""

'xstart=0.15\nxend=0.2\n\ncoor_xstart = int(xstart / h)\ncoor_xend = int(xend/h)\n\nplt.figure(figsize=(15, 10), dpi=240)\n#plt.title("Результат фильтрации по дискретизованным наблюдениям")\nplt.plot(np.arange(0,1,h)[coor_xstart:coor_xend], (X+1)[coor_xstart:coor_xend], label=\'Действительное значение\', zorder=2)\nplt.plot(np.arange(0,1,h)[coor_xstart:coor_xend], (argmax+1)[coor_xstart:coor_xend], label=\'Оценка\', zorder=1)\nplt.legend(loc=\'upper right\')\nplt.ylabel(\'Состояние МСП\')\nplt.xlabel(\'Время (год)\')\nplt.yticks([1, 2, 3, 4])\nplt.savefig("filtration_zoom_discrete.pgf", bbox_inches=\'tight\')\n#plt.show()'

In [23]:
xstart=0
xend=1

coor_xstart = int(xstart / h)
coor_xend = int(xend/h)
for i in range(1, dim_X+1):
    plt.figure(figsize=(12, 1), dpi=240)
    #plt.title("Результат фильтрации по дискретизованным наблюдениям ("+str(i)+"-состояние)")
    plt.plot(np.arange(0,1,h)[coor_xstart:coor_xend],np.equal(X, i-1)[coor_xstart:coor_xend], label='Действительное значение', zorder=2, lw=0.5)
    plt.plot(np.arange(0,1,h)[coor_xstart:coor_xend],np.equal(argmax,i-1)[coor_xstart:coor_xend], label='Оценка', zorder=1, lw=0.5)
    plt.plot(np.arange(0,1,h)[coor_xstart:coor_xend],X_pred[:,i-1][coor_xstart:coor_xend], label='Условная вероятность состояния', alpha=0.5, zorder=0, lw=0.5)
    #plt.legend(loc='upper right')
    #plt.xlabel('Время (год)')
    plt.savefig(str(i)+"state_discrete.pgf", bbox_inches='tight')
    #plt.show()

In [24]:
xstart=0.35
xend=0.4

coor_xstart = int(xstart / h)
coor_xend = int(xend/h)
for i in range(1, dim_X+1):
    plt.figure(figsize=(12, 1), dpi=240)
    #plt.title("Результат фильтрации по дискретизованным наблюдениям ("+str(i)+"-состояние)")
    plt.plot(np.arange(0,1,h)[coor_xstart:coor_xend],np.equal(X, i-1)[coor_xstart:coor_xend], label='Действительное значение', zorder=2, lw=0.5)
    plt.plot(np.arange(0,1,h)[coor_xstart:coor_xend],np.equal(argmax,i-1)[coor_xstart:coor_xend], label='Оценка', zorder=1, lw=0.5)
    plt.plot(np.arange(0,1,h)[coor_xstart:coor_xend],X_pred[:,i-1][coor_xstart:coor_xend], label='Условная вероятность состояния', alpha=0.5, zorder=0, lw=0.5)
    #plt.legend(loc='upper right')
    #plt.xlabel('Время (год)')
    plt.savefig(str(i)+"state_zoom_discrete.pgf", bbox_inches='tight')
    #plt.show()

In [25]:
np.sum(argmax == X) * h

0.7754521

In [26]:
np.sum(argmax == 1)*h

0.0031224

In [27]:
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [28]:
from sklearn.metrics import confusion_matrix
np.set_printoptions(suppress=True, precision=5)
print(confusion_matrix(X, argmax, labels=[0,1,2,3])*h)

[[0.49889 0.00078 0.00413 0.12021]
 [0.00288 0.00085 0.0024  0.00098]
 [0.00089 0.00149 0.05553 0.00925]
 [0.0049  0.      0.07663 0.22018]]


In [29]:
print(bmatrix(confusion_matrix(X, argmax, labels=[0,1,2,3])*h))

\begin{bmatrix}
  0.49889 & 0.00078 & 0.00413 & 0.12021\\
  0.00288 & 0.00085 & 0.0024 & 0.00098\\
  0.00089 & 0.00149 & 0.05553 & 0.00925\\
  0.0049 & 0. & 0.07663 & 0.22018\\
\end{bmatrix}


In [30]:
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + '\% & '.join(l.split()) + r'\%\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [31]:
print(bmatrix(confusion_matrix(X, argmax, labels=[0,1,2,3], normalize='true') * 100))

\begin{bmatrix}
  79.94854\% & 0.12565\% & 0.66118\% & 19.26463\%\\
  40.54491\% & 11.956\% & 33.69764\% & 13.80145\%\\
  1.32621\% & 2.21601\% & 82.68477\% & 13.773\%\\
  1.62408\% & 0.\% & 25.4\% & 72.97592\%\\
\end{bmatrix}
